# Spherical atmospheric grids

For atmospheric models spherical grids are quite common due to serious advantages. These grids are often defined by the Legendre polynomial for the distribution of the latitudes.
However throughout the literature different terminologies of these grids are used and the spherical grids are often not easy to compare to grid-point grids.
Here we offer a list of these spherical grids and their grid-spacing at the equator.

## Terminology

- **N:** Gaussian grid
- **T:** Truncation grid
- **T_L:** linear Truncation grid

## Notes

**Note 1:** *OFTEN when the grid is referenced people tend to drop the `_L`, so be aware of that.*

**Note 2:**  *This might not be 100% exact, as rounding errors might have occured.*

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from math import radians, cos, sin, asin, sqrt

# Source: https://gist.github.com/ajdawson/b64d24dfac618b91974f

"""Tools for working with Gaussian grids."""
from __future__ import (absolute_import, division, print_function)

import functools

import numpy as np
import numpy.linalg as la
from numpy.polynomial.legendre import legcompanion, legder, legval

def __single_arg_fast_cache(func):
    """Caching decorator for functions of one argument."""
    class CachingDict(dict):
    
        def __missing__(self, key):
            result = self[key] = func(key)
            return result
            
        @functools.wraps(func)
        def __getitem__(self, *args, **kwargs):
            return super(CachingDict, self).__getitem__(*args, **kwargs)
            
    return CachingDict().__getitem__


@__single_arg_fast_cache 
def gaussian_latitudes(n):
    """Construct latitudes and latitude bounds for a Gaussian grid.
    
    Args:
    
    * n:
        The Gaussian grid number (half the number of latitudes in the
        grid.

    Returns:
        A 2-tuple where the first element is a length `n` array of
        latitudes (in degrees) and the second element is an `(n, 2)`
        array of bounds.

    """
    if abs(int(n)) != n:
        raise ValueError('n must be a non-negative integer')
    nlat = 2 * n
    # Create the coefficients of the Legendre polynomial and construct the
    # companion matrix:
    cs = np.array([0] * nlat + [1], dtype=np.int)
    cm = legcompanion(cs)
    # Compute the eigenvalues of the companion matrix (the roots of the
    # Legendre polynomial) taking advantage of the fact that the matrix is
    # symmetric:
    roots = la.eigvalsh(cm)
    roots.sort()
    # Improve the roots by one application of Newton's method, using the
    # solved root as the initial guess:
    fx = legval(roots, cs)
    fpx = legval(roots, legder(cs))
    roots -= fx / fpx
    # The roots should exhibit symmetry, but with a sign change, so make sure
    # this is the case:
    roots = (roots - roots[::-1]) / 2.
    # Compute the Gaussian weights for each interval:
    fm = legval(roots, cs[1:])
    fm /= np.abs(fm).max()
    fpx /= np.abs(fpx).max()
    weights = 1. / (fm * fpx)
    # Weights should be symmetric and sum to two (unit weighting over the
    # interval [-1, 1]):
    weights = (weights + weights[::-1]) / 2.
    weights *= 2. / weights.sum()
    # Calculate the bounds from the weights, still on the interval [-1, 1]:
    bounds1d = np.empty([nlat + 1])
    bounds1d[0] = -1
    bounds1d[1:-1] = -1 + weights[:-1].cumsum()
    bounds1d[-1] = 1
    # Convert the bounds to degrees of latitude on [-90, 90]:
    bounds1d = np.rad2deg(np.arcsin(bounds1d))
    bounds2d = np.empty([nlat, 2])
    bounds2d[:, 0] = bounds1d[:-1]
    bounds2d[:, 1] = bounds1d[1:]
    # Convert the roots from the interval [-1, 1] to latitude values on the
    # interval [-90, 90] degrees:
    latitudes = np.rad2deg(np.arcsin(roots))
    return latitudes, bounds2d

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

N_start  = 1 
N_max    = 600
step     = 1
dist_Eq  = np.zeros(int((N_max-N_start)/step),dtype=int)
T        = dist_Eq.copy()
T_L      = dist_Eq.copy()
num_lats = dist_Eq.copy()

i=0
for N in np.arange(N_start,N_max,step):
    lat, _      = gaussian_latitudes(N)
    dist_Eq[i]  = int(haversine(0,lat[N],360/(N*4),lat[N]))
    T[i]        = int((2 * N -1))
    T_L[i]      = int(round((T[i] * 2 -1)/3)) #int((3 * T[i] + 1)/ 2)
    num_lats[i] = lat.shape[0]
    i=i+1
    
Grids = pd.DataFrame({'N': np.arange(N_start,N_max,step),\
                      'T': T,\
                      'T_L': T_L,\
                      'km at Eq':dist_Eq,\
                      'lats': num_lats},\
                     index=np.arange(N_start,N_max,step))

N_list   = np.array([96,216,512])
T_L_list = np.array([42,62,63,85,95,106,123,127,159,255,359,511])

dist_Eq  = np.zeros(int(len(N_list)),dtype=int)
T        = dist_Eq.copy()
T_L      = dist_Eq.copy()
num_lats = dist_Eq.copy()

i=0
for N in N_list:
    lat, _      = gaussian_latitudes(N)
    dist_Eq[i]  = int(haversine(0,lat[N],360/(N*4),lat[N]))
    T[i]        = int((2 * N -1))
    T_L[i]      = int(round((T[i] * 2 -1)/3)) #int((3 * T[i] + 1)/ 2)
    num_lats[i] = lat.shape[0]
    i=i+1

Grids_N_list = pd.DataFrame({'N': N_list,\
                      'T': T,\
                      'T_L': T_L,\
                      'km at Eq':dist_Eq,\
                      'lats': num_lats},\
                     index=N_list)

dist_Eq  = np.zeros(int(len(T_L_list)),dtype=int)
T        = dist_Eq.copy()
T_L      = dist_Eq.copy()
num_lats = dist_Eq.copy()

N_list_in= np.array((((3 * T_L_list + 1)/ 2) +1)/2,dtype=int)

i=0
for N in N_list_in:
    lat, _      = gaussian_latitudes(N)
    dist_Eq[i]  = int(haversine(0,lat[N],360/(N*4),lat[N]))
    T[i]        = int((2 * N -1))
    T_L[i]      = int(round((T[i] * 2 -1)/3)) #int((3 * T[i] + 1)/ 2)
    num_lats[i] = lat.shape[0]
    i=i+1

Grids_T_L_list = pd.DataFrame({'N': N_list_in,\
                      'T': T,\
                      'T_L': T_L,\
                      'km at Eq':dist_Eq,\
                      'lats': num_lats},\
                     index=N_list_in)

# Common model resolutions (out of CMIP6)

In [2]:
Grids_N_list

,N,T,T_L,km at Eq,lats
96,96,191,127,104,192
216,216,431,287,46,432
512,512,1023,682,19,1024


In [3]:
Grids_T_L_list

,N,T,T_L,km at Eq,lats
32,32,63,42,312,64
47,47,93,62,212,94
48,48,95,63,208,96
64,64,127,84,156,128
72,72,143,95,138,144
80,80,159,106,125,160
93,93,185,123,107,186
96,96,191,127,104,192
120,120,239,159,83,240
192,192,383,255,52,384


# an overview list

In [4]:
Grids[0:600:10]

,N,T,T_L,km at Eq,lats
1,1,1,0,7837,2
11,11,21,14,906,22
21,21,41,27,475,42
31,31,61,40,322,62
41,41,81,54,243,82
51,51,101,67,196,102
61,61,121,80,163,122
71,71,141,94,140,142
81,81,161,107,123,162
91,91,181,120,109,182


# an even more complete list

In [5]:
pd.options.display.max_rows = 999
Grids

,N,T,T_L,km at Eq,lats
1,1,1,0,7837,2
2,2,3,2,4687,4
3,3,5,3,3235,6
4,4,7,4,2457,8
5,5,9,6,1977,10
6,6,11,7,1653,12
7,7,13,8,1420,14
8,8,15,10,1244,16
9,9,17,11,1107,18
10,10,19,12,997,20
